<a href="https://colab.research.google.com/github/aeleraqi/NLP101/blob/main/NLP101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **تثبيت المكتبات الضرورية**

In [ ]:
pip install transformers

In [ ]:
pip install nltk

In [ ]:
pip install python-crfsuite

In [ ]:
pip install wordcloud

In [ ]:
pip install -U gensim

In [ ]:
import nltk
import string
import torch
import numpy as np
import logging
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import word_tokenize
from collections import Counter

In [ ]:
#Download the 'punkt' resource
nltk.download('punkt')

In [ ]:
#Download the 'averaged_perceptron_tagger' resource
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.download('stopwords')

# **النص محل التطبيق**

In [ ]:
# Example Arabic text
arabic_text = "تعرض اللاعب محمد الشناوي لاعب النادي الأهلي لإصابة شديدة الخطورة، نقل على أثرها إلى المستشفى، لتلقي العلاج الطبي، لكن تعافى قبل أن يصل إلى المستشفى! في تاريخ 09/12/2023"

# **تجزئة النص لوحدات صغيرة**

In [ ]:
# Tokenization using nltk
tokens = word_tokenize(arabic_text)
print("Tokenization:")
print(tokens)

Tokenization:
['تعرض', 'اللاعب', 'محمد', 'الشناوي', 'لاعب', 'النادي', 'الأهلي', 'لإصابة', 'شديدة', 'الخطورة،', 'نقل', 'على', 'أثرها', 'إلى', 'المستشفى،', 'لتلقي', 'العلاج', 'الطبي،', 'لكن', 'تعافى', 'قبل', 'أن', 'يصل', 'إلى', 'المستشفى', '!', 'في', 'تاريخ', '09/12/2023']


In [ ]:
tokens = nltk.word_tokenize(arabic_text)
print("Tokenization:")
print(tokens)

Tokenization:
['تعرض', 'اللاعب', 'محمد', 'الشناوي', 'لاعب', 'النادي', 'الأهلي', 'لإصابة', 'شديدة', 'الخطورة،', 'نقل', 'على', 'أثرها', 'إلى', 'المستشفى،', 'لتلقي', 'العلاج', 'الطبي،', 'لكن', 'تعافى', 'قبل', 'أن', 'يصل', 'إلى', 'المستشفى', '!', 'في', 'تاريخ', '09/12/2023']


# **تحديد الكلمات الأكثر تكرارًا**

In [ ]:
# Calculate word frequencies
word_freq = Counter(tokens)

arabic_text = "تعرض اللاعب محمد الشناوي لاعب النادي الأهلي لإصابة شديدة الخطورة، نقل على أثرها إلى المستشفى، لتلقي العلاج الطبي، لكن تعافى قبل أن يصل إلى المستشفى! في تاريخ 09/12/2023"

# Find the most frequent word
most_freq_word = word_freq.most_common(2)

print("Most frequent word:")
print(most_freq_word)

Most frequent word:
[('إلى', 2)]


# **تحديد علامات الترقيم في النص**

In [ ]:
#Identify punctuation marks
punctuation_marks = [token for token in tokens if token in string.punctuation]

arabic_text = "تعرض اللاعب محمد الشناوي لاعب النادي الأهلي لإصابة شديدة الخطورة، نقل على أثرها إلى المستشفى، لتلقي العلاج الطبي، لكن تعافى قبل أن يصل إلى المستشفى! في تاريخ 09/12/2023"

print("Punctuation marks:")
print(punctuation_marks)

Punctuation marks:
['!']


# **تحديد أسماء الكيانات في النص**

In [ ]:
!pip install -q transformers==4.29 datasets==2.14.5 sentencepiece==0.1.99

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

import numpy as np
import logging
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model_cp = "marefa-nlp/marefa-ner"
custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                 "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                 "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

tokenizer = AutoTokenizer.from_pretrained(model_cp)
model = AutoModelForTokenClassification.from_pretrained(model_cp, num_labels=len(custom_labels))

In [ ]:
def _extract_ner(text: str, model: AutoModelForTokenClassification,
                 tokenizer: AutoTokenizer, start_token: str="▁"):
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
    tokenized_sentences = tokenized_sentence['input_ids'].numpy()

    with torch.no_grad():
        output = model(**tokenized_sentence)

    last_hidden_states = output[0].numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":

            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])

        prev_label = label

    # group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]

    return ents

In [ ]:
def _extract_ner(text: str, model: AutoModelForTokenClassification,
                 tokenizer: AutoTokenizer, start_token: str="▁"):
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
    tokenized_sentences = tokenized_sentence['input_ids'].numpy()

    with torch.no_grad():
        output = model(**tokenized_sentence)

    last_hidden_states = output[0].numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":

            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])

        prev_label = label

    # group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]

    return ents

# **التجربة والنتيجة**

In [ ]:
samples = ["تعرض اللاعب محمد الشناوي لاعب النادي الأهلي لإصابة شديدة الخطورة، نقل على أثرها إلى المستشفى، لتلقي العلاج الطبي، لكن تعافى قبل أن يصل إلى المستشفى! في تاريخ 09/12/2023"]

# [optional]
samples = [ " ".join(word_tokenize(sample.strip())) for sample in samples if sample.strip() != "" ]

for sample in samples:
    ents = _extract_ner(text=sample, model=model, tokenizer=tokenizer, start_token="▁")

    print(sample)
    for ent in ents:
        print("\t",ent["token"],"==>",ent["label"])
    print("========\n")

تعرض اللاعب محمد الشناوي لاعب النادي الأهلي لإصابة شديدة الخطورة، نقل على أثرها إلى المستشفى، لتلقي العلاج الطبي، لكن تعافى قبل أن يصل إلى المستشفى ! في تاريخ 09/12/2023
	 اللاعب ==> job
	 محمد الشناوي ==> person
	 لاعب ==> job
	 النادي الأهلي ==> organization
	 09/12/2023 ==> time

